In [41]:
import csv
import pandas
import pathlib

In [63]:
extractor = {
    "lbp": [59],
    "surf64": [128, 256, 257],
    "surf128": [128, 256, 513],
    "mobilenetv2": [128, 256, 512, 1024, 1280],
    "resnet50v2": [128, 256, 512, 1024, 2048],
    "vgg16": [128, 256, 512]
}

list_classifiers = ["DecisionTreeClassifier", "KNeighborsClassifier", "MLPClassifier", "RandomForestClassifier", "SVC"]
list_dimensions = [256, 400, 512]
list_patch = ["patch=3", "patch=5", "patch=7"]
list_orientation_patch = ["horizontal", "vertical"]
list_type_mask = ["_manual_", "_unet_"]
list_info = ["_mean_", "_std_"]
list_dataset = ["unet", "iwssip", "matlab", "photoshop"]

## mean

In [64]:
index = [k + s + str(d) for s in list_info for k, v in extractor.items() for d in v]
index_time = [k + "_" + str(d) for k, v in extractor.items() for d in v]
csv_per_dataset = {d: {"sheet_mean": {"columns": [c + "_iwssip_" + str(d) for c in list_classifiers for d in list_dimensions], "index": index}, "sheet_time": {"columns": [c + "_iwssip_" + str(d) for c in list_classifiers for d in list_dimensions], "index": index_time}} for p in list_patch for o in list_orientation_patch for d in list_dataset if d.lower() != "iwssip"}

for s in csv_per_dataset.keys():
    for c in list_classifiers:
        for d in list_dimensions:
            csv_per_dataset[s]["sheet_mean"]["columns"].append(c + "_" + s + "_" + str(d))

for p in csv_per_dataset.keys():
    csv_per_dataset[p]["sheet_mean"] = pandas.DataFrame(columns=sorted(csv_per_dataset[p]["sheet_mean"]["columns"]), index=sorted(csv_per_dataset[p]["sheet_mean"]["index"]))
    csv_per_dataset[p]["sheet_time"] = pandas.DataFrame(columns=sorted(csv_per_dataset[p]["sheet_time"]["columns"]), index=sorted(csv_per_dataset[p]["sheet_time"]["index"]))

## time

In [72]:
list_files = list([file for file in pathlib.Path("./r/GRAYSCALE").rglob("mean.csv") if file.is_file()])

patch = "patch=3"
orientation = "horizontal"
dataset = "lbp"

for file in list_files:
    data = pandas.read_csv(file, sep=";", header=None, index_col=0).squeeze()

    get_dataset = list(filter(lambda x: x.lower() in str(file).lower(), list_dataset))[0]
    get_dim = list(filter(lambda x: str(x).lower() in str(file).lower(), list_dimensions))[0]
    filename = str(file).replace(str(get_dim), "", 1)
    get_extractor = list(filter(lambda x: x.lower() in str(filename).lower(), list(extractor.keys())))[0]
    get_classifier = list(filter(lambda x: x.lower() in str(filename).lower(), list_classifiers))[0]
    get_patch = list(filter(lambda x: x.lower() in str(filename).lower(), list_patch))
    get_patch = "patch=0" if len(get_patch) == 0 else get_patch[0]
    get_orientation_patch = list(filter(lambda x: x.lower() in str(filename).lower(), list_orientation_patch))
    get_orientation_patch = "inteiro" if len(get_orientation_patch) == 0 else get_orientation_patch[0]

    get_features = list(filter(lambda x: str(x) in str(filename), extractor[get_extractor]))
    for f in get_features:

        # print(get_dataset, f, get_dim, get_extractor, get_classifier)
        # print(file)

        if get_patch == patch and get_orientation_patch == orientation and get_dataset == dataset:
            if get_dataset=="iwssip":
                for c in csv_per_dataset.keys():
                    index_df = get_extractor + "_mean_" + str(f)
                    column_df = get_classifier + "_" +  get_dataset + "_" + str(get_dim)
                    csv_per_dataset[c]["sheet_mean"].loc[index_df, column_df] = float(data["mean_sum"])

                    index_df = get_extractor + "_std_" + str(f)
                    print(f"aa{index_df, column_df}")
                    csv_per_dataset[c]["sheet_mean"].loc[index_df, column_df] = "±" + data["std_sum"]

                    index_df_time = get_extractor + "_" + str(f)
                    csv_per_dataset[c]["sheet_time"].loc[index_df_time, column_df] = data["mean_time_millisec"]

            else:
                index_df = get_extractor + "_mean_" + str(f)
                column_df = get_classifier + "_" +  get_dataset + "_" + str(get_dim)
                # print(index_df, column_df)
                csv_per_dataset[get_dataset]["sheet_mean"].loc[index_df, column_df] = float(data["mean_sum"])

                index_df = get_extractor + "_std_" + str(f)
                print(f"bb{index_df, column_df}")
                csv_per_dataset[get_dataset]["sheet_mean"].loc[index_df, column_df] = "±" + data["std_sum"]

                index_df_time =  get_extractor + "_" + str(f)
                csv_per_dataset[get_dataset]["sheet_time"].loc[index_df_time, column_df] = data["mean_time_millisec"]


In [66]:
for p in csv_per_dataset.keys():
    csv_per_dataset[p]["sheet_mean"].to_csv(f"main_mean_{p}.csv", decimal=",", sep=";", quoting=csv.QUOTE_ALL, mode="w")
    csv_per_dataset[p]["sheet_time"].to_csv(f"main_time_{p}.csv", decimal=",", sep=";", quoting=csv.QUOTE_ALL, mode="w")